In [ ]:
import numpy as np
from pathlib import Path
from tqdm.autonotebook import tqdm
from copy import deepcopy

from pyintdem.models.band import Band
from pyintdem import read_file

In [ ]:
fdir = Path("../IntertidalDEM_rev2023/Shorelines")
tiles = list(fdir.glob(pattern="*/"))
tiles

In [ ]:
def list_bw_files(tile_dir, fname="bw_clean_0.5_3.0.tif"):
    images = list(tile_dir.glob("*/"))

    bw_files = [image / fname for image in images]
    bw_files = list(filter(lambda fn: fn.exists(), bw_files))
    return bw_files

def compute_frequency(bw_files):
    for i, bw_file in enumerate(tqdm(bw_files)):
        
        img_band = read_file(bw_file)

        if i == 0:
            count_band = np.logical_not(np.isnan(img_band.data)).astype(int)
            freq_band = img_band
        else:
            count_band = count_band + np.logical_not(np.isnan(img_band.data)).astype(int)
            freq_band = freq_band.nan_sum(img_band)

    
    count_band = Band(data=count_band, geotransform=freq_band.geotransform, projection=freq_band.projection)
    freq_band = freq_band / count_band.astype(float)

    return freq_band, count_band

def is_intertidal(freq_band, lower=0.1, upper=1):
    intertidal = deepcopy(freq_band)
    intertidal.data[intertidal.data>=upper] = np.nan
    intertidal.data[intertidal.data <= lower] = np.nan
    intertidal.data[np.logical_not(np.isnan(intertidal.data))] = 1
    return intertidal

In [ ]:
def process_tile(tile, dir_freq, dir_int_mask, lower_limit, upper_limit):
    bw_files = list_bw_files(tile_dir=tile, fname="bw_clean_0.5_3.0.tif")
    tile_name = tile.name
    
    freq_band, count_band = compute_frequency(bw_files=bw_files)
    freq_band.to_geotiff(dir_freq / f"{tile_name}_freq.tif")
    count_band.to_geotiff(dir_freq / f"{tile_name}_count.tif")

    intertidal_band = is_intertidal(freq_band, lower=lower_limit, upper=upper_limit)
    intertidal_band.to_geotiff(dir_int_mask / f"{tile_name}_{lower_limit}_{upper_limit}.tif")

def process_all_tiles():
    dir_freq = Path("./Frequency")
    dir_int_mask = Path("./Intertidal")

    lower_limit = 0.1
    upper_limit = 0.9

    for fdir in [dir_freq, dir_int_mask]:
        if not fdir.exists():
            fdir.mkdir()

    for tile in tiles:
        try:
            process_tile(
                tile=tile, 
                dir_freq=dir_freq, 
                dir_int_mask=dir_int_mask, 
                lower_limit=lower_limit, 
                upper_limit=upper_limit)
        except:
            print(f"{tile} failed")

process_all_tiles()